<b>Neutron stars are laboratories for physics in conditions more extreme than anything we can do here on Earth.</b> There are mysteries hidden in these stars and we would learn a ton if only we could see their interiors in detail. One of the biggest open questions of physics today is: how does the strong nuclear force work? This is the key for describing nuclei from first principles, and the details of such a fundamental force are important in so many areas of physics and astronomy.

<b>Comparing theoretical neutron stars against real ones will verify whether a nuclear equation of state is correct.</b>

# Let's start!

There are some code libraries that we need to import.

- <b>Numpy</b> contains methods and objects that let us do mathematical operations easily and quickly.
- We will use <b>bisect</b> to invert equations numerically.
- <b>Matplotlib</b> lets us draw graphics.

In [ ]:
import numpy as np
import bisect
import matplotlib.pyplot as plt

There are various <b>physical constants</b> that we are going to need in our calculations.

In [ ]:
c = 2.99792458e10 # speed of light, cm/s
h = 6.6260755e-27 # Planck constnat, erg*s
G = 6.67259e-8 # gravitational constant, cm^3/g/s
e = 4.8032068e-10 # charge of an electron, esu
me = 9.1093897e-28 # mass of an electron, g
mB = 1.66057e-24 # mass of a baryon, g
Msun = 1.99e33 # mass of the sun, g
Rsun = 6.96e10 # radius of the sun, cm
km = 1e5 # cm

# Ingredient 1: Equation of State

The first step of the puzzle is the equation of state, which <b>relates the pressure with the density</b>. Most neutron stars that we see have had a lot of time to cool down, which allows us to assume that they are cold enough that we don't have to think about their temperature. However, neutrons are so dense that we have to treat them as degenerate fermions, and because of that they can have a lot of kinetic energy. In fact, they can have so much energy that they become relativistic.

Find Equations 11-15 in Ref. [1] (reproduced from an article by Chandrasekhar [2]), which show how to describe this process analytically. Note the notation and nomenclature used in said article:
- The <b>total density</b> $\rho = \rho_\mathrm{mass} + \rho_\mathrm{energy}$ is the sum of the mass and energy density, respectively (units of erg/cm$^3$).
- The <b>mass density</b> is $\rho_\mathrm{mass} = \mu_0 N/V$ (g/cm$^3$).
- $N$ and $V$ are the number of particles and the volume, respectively. However, this is not too important, since they are never actually used in the final equations.

There is no closed form solution for the equations, and because of that we will use a parameterized form of the equations. The symbol $t$ does not mean anything physical - it is just a useful tool for solving the equations.


- [1] <b>Oppenheimer, J. R., & Volkoff, M.</b> ”On Massive Neutron Star Cores.” Physical Review, 55 (1939). https://journals.aps.org/pr/pdf/10.1103/PhysRev.55.374
- [2] <b>Chandrasekhar, S.</b> ”The highly collapsed configurations of a stellar mass (Second paper).” Monthly Notices to the Royal Astronomical Society, 95 (1935). http://articles.adsabs.harvard.edu/pdf/1935MNRAS..95..207C

## [5 points] Function for interpolating data

The physics of dense matter involves quite complicated equations. <b>We will need to solve equations that are not possible to solve analytically.</b> Here we will write a function to interpolate a mathematical function in order to make our future work easier.

<b>Uncomment the function and execute the code</b> to try the interpolation function.

In [ ]:
# xIn - x where we want to obtain a value of y
# xGrid - an array of x values
# yGrid - an array of y values evaluated at at the x locations
#def interpolate(xIn, xGrid, yGrid):
#    # get the upper indices
#    i = bisect.bisect_right(xGrid, xIn, lo=0, hi=len(xGrid))
#
#    # check for degenerate case
#    yRight    = yGrid[i  ];
#    xRight    = xGrid[i  ];
#    yLeft  = yGrid[i-1];
#    xLeft  = xGrid[i-1];
#    if(yRight == yLeft):
#        return yRight;
#
#    # do the interpolation
#    slope = (yRight-yLeft) / (xRight-xLeft);
#    yOut = yLeft + (xIn-xLeft)*slope;
#    return yOut;

xGrid = np.linspace(0,1,10)
yGrid = xGrid**3
xTest = 0.5
yTest = interpolate(xTest, xGrid, yGrid)
plt.plot(xGrid, yGrid,marker="+",label="yGrid")
plt.scatter(xTest,yTest, label="yTest")
plt.legend()
print("yTest =", yTest)

## [30 points] The Equation of State Class: calculate the pressure and internal energy at any mass density
1. <b>Find the equation of state of relativistic dense matter in the article by Oppenheimer y Volkoff.</b> What type of equation is it? If we have the mass density, can we calculate the pressure analytically? If we have the pressure, can we calculate the mass density analytically?
1. In the constructor \_\_init\_\_():
    1. <b>Uncomment the first block of code and print the array 'MassDensity'.</b> We are going to find all of the properties of the material at each of these values of mass density. What is the range of densities that we are looking at? How dense are atomic nuclei?
    1. <b>Write the expression for calculating the constant K</b> (Equation 13 in [1]). Verify that it has units of energy density.
    1. <b>Use the array to calculate an array of momenta ($\hat{p}$, Equation 15 in [1]). Verify that it has units of momentum.
    1. <b>Use the array of momenta to calculate an array of values of t</b> (Equation 14 in [1]). Verify that it is dimensionless.
    1. <b>Use the array of $t$ values to calculate an array of pressure values</b> (called Pressure, Equation 12) and total density (called TotalDensity, Equation 11). Verify that both quantities have units of energy density.
    1. <b>Uncomment the last block of code</b> to save the arrays in the EOS class.
1. Now we have all of the equations, but how can we calculate the pressure if we only know the density? <b>Write the functions calcPressure y calcEnergyDensity</b>, using calcMassDensity as an example.
1. We will also need the total density, including mass and energy. <b>Write the function calcTotalDensity</b>. Make sure that the units of the returned quantity are units of mass density and not energy density.

In [ ]:
class EOS:
    def __init__(self, MassDensityMin, MassDensityMax, npoints):
        # create an array of mass densities
        # Step 2A
        #log10MassDensityMin = np.log10(MassDensityMin)
        #log10MassDensityMax = np.log10(MassDensityMax)
        #dlog10MassDensity = (log10MassDensityMax - log10MassDensityMin) / (npoints-1)
        #self.log10MassDensity = np.array([log10MassDensityMin + i*dlog10MassDensity for i in range(npoints)])
        #MassDensity = 10**self.log10MassDensity # g/ccm

        # Define constants needed for Ecuacion 13 in the Oppenheimer article
        # Step 2B
        #K = IMPLEMENT_ME; # erg/ccm
        
        # Create an array of phat values
        # Step 2C
        #phat = IMPLEMENT_ME # g*cm/s

        # Create an array of values of t
        # Step 2D
        #t = IMPLEMENT ME
        
        # Create arrays of pressure and energy density
        # Step 2E
        #TotalDensity  = IMPLEMENT ME # erg/ccm
        #Pressure      = IMPLEMENT ME # erg/ccm
        #EnergyDensity = IMPLEMENT ME # erg/ccm

        # Create logarithmic arrays for interpolation
        # Step 2F
        #self.log10MassDensity   = np.log10(MassDensity)
        #self.log10Pressure      = np.log10(Pressure)
        #self.log10EnergyDensity = np.log10(EnergyDensity)
        
    # Interpolate pressure based on the mass density
    # Step 3
    #def calcPressure(self,MassDensity): # erg/ccm
    #    x = IMPLEMENT ME
    #    xgrid = IMPLEMENT ME
    #    ygrid = IMPLEMENT ME
    #    return IMPLEMENT ME
        
    # Interpolate mass density given the pressure
    def calcMassDensity(self,Pressure): # g/ccm
        #x = np.log10(Pressure)
        #xgrid = self.log10Pressure
        #ygrid = self.log10MassDensity
        return 10**interpolate(x, xgrid, ygrid)

    # Interpolate the energy density given the mass density
    # Step 3
    def calcEnergyDensity(self, Pressure): # erg/ccm
        #x = IMPLEMENT ME
        #xgrid = IMPLEMENT ME
        #ygrid = IMPLEMENT ME
        #return IMPLEMENT ME
    
    # Calculate the total density based on the pressue
    # Step 4
    def calcTotalDensity(self, Pressure): # g/ccm
        #return IMPLEMENT ME
    
# Initialize the equation of state.
# We just care about densities between 1e10 and 1e20 g/ccm,
# but we will plot a wider range to see how it looks.
# Calculate 200 points within the EOS class.
eos = EOS(1e9, 1e20, 200)

# Make a plot that shows the equation of state
pres,  = plt.loglog(10**eos.log10MassDensity, 10**eos.log10Pressure, label="Pressure")
edens, = plt.loglog(10**eos.log10MassDensity, 10**eos.log10EnergyDensity, label="Energy Density")

# Draw the legend
plt.legend()

# Draw the axis labels
plt.xlabel("Mass Density (g cm$^{-3}$)")
plt.ylabel("Pressure and Energy Density (erg cm$^{-3}$)")
plt.show()

Now we have a description of dense matter! Put we need to use it to calculate properties of neutron stars. <b>Comparing our theoretical neutron stars against real ones will verify whether our equation of state is correct.</b>

# [40 points] Ingredient 2: Equations of Stellar Structure

We are not going to re-derive the equations of relativistic stellar structure here. Instead, we will find the relevant equations in the scientific literature and convert them into code.

The important quantities are:
- $\rho_\mathrm{tot}$: the total density of mass and energy together
- $M(r)$: the mass internal to a radius $r$
- $P$: the pressure

Now, the work:

1. <b>Find the equations of relativistic star structure </b>in the article by Oppenheimer and Volkovv</br>
https://journals.aps.org/pr/pdf/10.1103/PhysRev.55.374

1. Before implementing the equations, <b>verify that we can recover the equations of regular star structure </b>in the limit of a non-relativistic star (i.e., $P << \rho_\mathrm{tot}$).

1. Then, <b>write the code to calculate the derivatives of pressure and total internal mass</b>.

1. <b>Execute the code</b> and verify that the numbers coincide with the expected values (they were calculated previously).

Now we have the differential equations that we are going to use to construct theoretical neutron stars. <b>Comparing our theoretical neutron stars with real ones will verify whether this equation of state is correct.</b>

In [ ]:
# How quickly does the pressure change with radius
# based on the radius r, the total density, the pressure, and the total mass internal to the radius r
# Step 3
def dPressure_dr(r, TotalDensity, Pressure, TotalInternalMass): # cm, g/ccm, erg/ccm, g
    #result = IMPLEMENT ME # erg/ccm/cm
    return result
    
# How quicly does the total internal mass change with radius
# based on the radius r and the total density at that radius
# Step 3
def dTotalInternalMass_dr(r, TotalDensity): # cm, g/ccm
    #result = IMPLEMENT ME # g
    return result

r = 1e5 # cm
MassDensity = 1e12 # g/ccm
Pressure = eos.calcPressure(MassDensity) # erg/ccm
TotalInternalMass = 1e33 # g

print("dPressure_dr expected: ", 1.3770551299991524e+28)
print("dPressure_dr calculated: ", dPressure_dr(r, MassDensity, Pressure, TotalInternalMass))
print()
print("dTotalInternalMass_dr expected: ", 1.25663706144e+23)
print("dTotalInternalMass_dr calculated: ",dTotalInternalMass_dr(r, MassDensity))

We have the differential equations in the computer! We are going to <b>integrate the equations approximately</b> with the method of finite differences. First, notice that a derivative is approximated by a fraction of differences:

$\frac{df}{dr} \approx \frac{\Delta f}{\Delta r}$

So, if we know the pressure $P$ and internal mass $M$ at the radius $r$, we can advance a step $dr$ using

$P(r+dr) \approx P(r) + dr \frac{dP}{dr}$

$M(r+dr) \approx M(r) + dr \frac{dM}{dr}$

<b>Write a function that advances $P$ y $M$ by one step $dr$.</b>
- `r0` is the radius r at the beginning of the step
- `Pressure0` is the pressure at r0
- `TotalInternalMass0` es la total internal mass inside of r0
- `dr` is the size of the step
- `eos` is the equation of state class that we wrote above

In [ ]:
# Advance the pressure and total internal mass outward by a step dr
def integrate(r0, Pressure0, TotalInternalMass0, dr, eos):

    # Calculate the total density at radius r0 using the EOS class
    #TotalDensity0 = IMPLEMENT ME
    
    # Calculate the derivatives at radius r0 using the functions you wrote above
    #dPdr = IMPLEMENT ME
    #dMdr = IMPLEMENT ME

    # Advance the radius, pressure, and total internal mass
    #r1 = IMPLEMENT ME
    #Pressure1          = IMPLEMENT ME
    #TotalInternalMass1 = IMPLEMENT ME

    return r1, Pressure1, TotalInternalMass1

Finally, we will write <b>the function for constructing a complete neutron star</b>. We will choose a central density and then integrate the equations until we arrive at the surface. All we need to do is use our function `integrate` to take many steps.

- `dr` is the size of each step, starting from $r=0$. Smaller values of `dr` make for a more precise solution.
- `CentralMassDensity` is the mass density at the center of the star. We will choose a value, and the rest of the star will be calculated.
- `MassDensity_stop` is the mass density at which we will stop the outward integration. This can be understood as the density which we choose to define "the surface".
- `print_values` allows us to print values at every step

1. <b>Uncomment the first block of code.</b> These are the conditions at the center of the star, determined by the `CentralMassDensity`
1. <b>Write the total mass inside the first step. </b>This is just basic physics - don't over-complicate it
1. The while loop will keep taking steps until the mass density is sufficiently small.
    1. <b>Use the function `integrate`</b> to take a step in r, Pressure, and TotalInternalMass
    1. <b>Use the function `eos.calcMassDensity`</b> to obtain the mass density after the step.
    1. <b>Which values are we printing at every step? What do they mean?</b>
1. Execute the code below to construct a star with central mass density of $10^{15}\,\mathrm{g/cm}^3$. <b>What are the radius and total mass of the resulting neutron star?</b>

In [ ]:
def calculateRadiusAndMass(dr, CentralMassDensity, MassDensity_stop, eos, print_values=False):
    # Initial conditions (at the center of the star). Take the first step
    # Step 1
    r = dr
    MassDensity = CentralMassDensity
    Pressure = eos.calcPressure(MassDensity) # erg/ccm
    TotalDensity = eos.calcTotalDensity(Pressure) # g/ccm
    
    # Calculate the internal mass after the first step
    # Step 2
    #TotalInternalMass = IMPLEMENT ME # g
    
    # Take radial steps until the mass density is sufficiently small
    i=0
    while MassDensity > MassDensity_stop:

        # Advance the radius, pressure, and total internal mass by a step dr using the function integrate()
        # Step 3A
        #r, Pressure, TotalInternalMass = IMPLEMENT ME

        # Calculate the mass density at the new radius using the EOS class
        # Step 3B
        #MassDensity = IMPLEMENT ME
        
        # Print the radius (km), mass density (g/ccm), and the total internal mass (Msun)
        if print_values and i%1000==0:
            print(str(i)+"\t"+str(r/km)+"\t"+str(MassDensity)+"\t"+str(TotalInternalMass/Msun))
        i = i+1
    if print_values:
        print(str(i)+"\t"+str(r/km)+"\t"+str(MassDensity)+"\t"+str(TotalInternalMass/Msun))
        
    return r, TotalInternalMass



print("Step   Radius(km)   MassDensity(g/ccm)   TotalInternalMass(Msol)")

# Constants that determine the quality of the calculation
dr = 10 # cm
MassDensity_stop = 1e10 # g/ccm

# Try to calculate the mass and radius of just one neutron star
CentralMassDensity = 1e15
R, M = calculateRadiusAndMass(dr, CentralMassDensity, MassDensity_stop, eos, True)

# [5 points] Numerical Error

The parameters  `dr` y `MassDensity_stop` were chosen by us. <b>The properties of the neutron stars we calculate should be independent of our choices, else the numbers we get are just artifacts of our finite-step approximation and don't provide any meaningful information about the solution to the physical equations.</b>.
1. <b>Calculate the radius and mass with different values of these quantities.</b> How large can we make `dr` before the results significantly change?

In [ ]:
dr = 10 # cm
MassDensity_stop = 1e10 # g/ccm
CentralMassDensity = 1e15
R, M = calculateRadiusAndMass(dr, CentralMassDensity, MassDensity_stop, eos, False)
print("radius =",R/1e5,"km")
print("mass =",M/Msun,"Msun")

# [ 5 points] That was one star. What kinds of neutron stars are possible?

Now we are going to calculate the mass and radius of various neutron stars with different central densities.
1. What is the array `CentralDensities` for?
1. What do the quantities `Rlist` y `Mlist` mean?
1. <b>Execute the code below to construct many neutron stars.</b> This could take a few minutes.

In [ ]:
print("CentralMassDensity(g/ccm)   Radius(km)   Mass(Msun)")

# Calculate the mass and radius of many neutron stars and make a plot
CentralMassDensity0 = 1e14 # g/ccm
CentralMassDensity1 = 1e17 # g/ccm
NumberOfStars = 50

# Create an array of central densities. Each element represents a different star.
CentralDensities = np.exp(np.linspace(np.log(CentralMassDensity0), np.log(CentralMassDensity1), NumberOfStars))

# Create empty lists of radii and masses
Rlist = []
Mlist = []

# Loop over the central densities and obtain a radius and mass for each
for iStar in range(NumberOfStars):
    CentralMassDensity = CentralDensities[iStar]
    R,M = calculateRadiusAndMass(dr, CentralMassDensity, MassDensity_stop, eos, False)
    Rlist.append(R/km)
    Mlist.append(M/Msun)
    print(CentralMassDensity, R/km, M/Msun)
    
# Draw a plot of the data
plt.grid()
plt.xlabel("Radius (km)")
plt.ylabel("Mass (Msun)")
plt.scatter(Rlist, Mlist)

# [15 points] Finally, the science!
1.  <b>Understand the plot</b>
    1. Which point corresponds to the smallest central density?
    1. Which star is the largest?
    1. Why can't we make a neutron star with an arbitrary mass and radius?
    1. What is the maximum neutron star mass we can make with our theoretical equation of state?
    
1. <b>Compare our results to real neutron stars.</b> Go to https://stellarcollapse.org/index.php/nsmasses.html. You can see that all of the neutron star masses have error bars of very different sizes.
    1. What is the largest mass with small error bars?
    1. Which measured masses are compatible with those predicted by our theoretical equation of state?
    
1. Go to the article at https://arxiv.org/abs/1912.05703 and look at Figure 1. NICER is an instrument on the International Space Station, and with it scientists are trying to infer the mass and radius of neutron stars more precisely in order to better understand the equation of state. This figure shows the uncertainty band in neutron star masses and radii based on measurements of the light coming from the neutron stars. <b>Are our results within the measurement errors indicated by the figure?</b>

1. Go to the article at https://arxiv.org/abs/1701.02752 and look at Figure 2. The figure shows the results of several more complicated theories of the equation of state. <b>How many of those theories could be correct?</b>

1. <b>Open questions</b>:
    1. How interesting are discoveries in this field to the general public?
    1. What are we missing from our theories?
    1. Is our theory of the equation of state wrong or just a piece of a more complete theory?
    1. How could somebody contribute to this field of research?